In [2]:
# import sys
# sys.path.append('/home/xykong/diffusion-text-generation/')

In [ ]:
# sys.path

In [25]:
import pickle
from torch.utils.tensorboard import SummaryWriter
from model_arch.tokenizer import load_tokenizer
import torch
# from pathlib import Path

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'

In [26]:
# best_model_fp = f'models/better/model_best_epoch_22400_min_val_loss_0.027400000020861626.pkl' # warm up + llrd, dropout 0.1, bs 20
# best_model_fp = 'models/better/model_best_epoch_22400_min_val_loss_0.028599999845027924.pkl' # warm up + llrd, dropout 0.2, bs 20
# best_model_fp = 'models/better/model_best_epoch_20600_min_val_loss_0.028699999675154686.pkl' # dropout 0.2, bs 15
# best_model_fp = 'models/better/model_best_epoch_20600_min_val_loss_0.026000000536441803.pkl' # warm up, dropout 0.1, bs 15
best_model_fp = 'models/final/model_best_epoch_22400_min_val_loss_0.025299999862909317.pkl'
with open(best_model_fp, 'rb') as handle:
    best_model = pickle.load(handle)

In [27]:
for param in best_model.parameters():
    param.requires_grad = False

# Comparing diffusion models (by # steps)

In [28]:
from model_arch.gaussian_diffusion import get_named_beta_schedule, SpacedDiffusion
from model_arch.tokenizer import load_tokenizer
from transformers import set_seed
from model_arch.sampling import *

In [30]:
noise_schedule='sqrt'
predict_xstart=True
rescale_timesteps=True
regular_data_dir='data'
seed=102
config_name='bert-base-uncased'

set_seed(seed)
tokenizer = load_tokenizer('shakespeare', config_name)

shakespeare


In [31]:
diffusion_100 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 100),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_500 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 500),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_1000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 1000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

diffusion_2000 = SpacedDiffusion(
    betas=get_named_beta_schedule(noise_schedule, 2000),
    rescale_timesteps=rescale_timesteps,
    predict_xstart=predict_xstart,
)

### With 100 diffusion steps

In [32]:
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_100, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128, 
                                                           show_intermediate_results=False)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 2082.54 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 5
})
RAM used: 2082.54 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 5
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14472, 102, 94, 451, 9, 41, 2679, 110, 457, 19200, 105, 1079, 367, 23844, 12, 3]
RAM used: 2084.09 MB


merge and mask:   0%|          | 0/5 [00:00<?, ? examples/s]

RAM used: 2084.09 MB


padding:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 5
}) padded dataset
RAM used: 2084.09 MB
RAM used: 2084.09 MB
### End of reading iteration...


  0%|          | 0/100 [00:00<?, ?it/s]

In [33]:
word_lst_source, word_lst_recover

(["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]"],
 ['[CLS] nor best, from being. [SEP]',
  '[CLS] not i am, good, not wife, very wife, [SEP]',
  "[CLS] o queen my lord, he see'd their it, as and and return, wife [SEP]",
  '[CLS] do man, i was me, she one to the young more wife,, one [SEP]',
  '[CLS] she,, sir, is of so. [SEP]'])

### With 500 diffusion steps

In [12]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]
word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_500, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['life has meanings.', 'call her forth to me.'] ['', '']
RAM used: 2050.26 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 5
})
RAM used: 2050.26 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 5
})
### tokenized_datasets...example [2, 451, 779, 14765, 12, 3]
RAM used: 2052.77 MB


merge and mask:   0%|          | 0/5 [00:00<?, ? examples/s]

RAM used: 2052.80 MB


padding:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 5
}) padded dataset
RAM used: 2052.86 MB
RAM used: 2052.86 MB
### End of reading iteration...


  0%|          | 0/500 [00:00<?, ?it/s]

In [13]:
word_lst_source, word_lst_recover

(['[CLS] life has meanings. [SEP] [SEP]',
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]"],
 ['[CLS] i it i will, but this better my that to the i being. [SEP]',
  "[CLS] and not'll i will [SEP]",
  '[CLS] no, sir wife, who. [SEP]',
  '[CLS] with is so, be shows in, [SEP]',
  "[CLS] indeed nay'd love see face from. [SEP]"])

In [14]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

spetbered jellthoriz waftsdisrehensiveurnishts ideas innov hostiliuslot erbl kild crispin moral shorten bunting memoriz indifferency quoif [PAD]allinewinkually remaineth neigh [PAD] escapesoons disc artist cots resol overleather shards penth dividable [PAD] li [PAD] mu repugnant mildest afflicts intermit disprov ermengare [PAD]met includes conspirant fainted scrap gentlehearted vest demoisell importeth proffers flapdragons [PAD] glose revol chart tem incensement crossest wracks calipolis predecease [PAD] loathsomeness cerns handle ov ascends preached throbs lapis tragedian [PAD]imable favoured intercessors deedless florent knavery buzzers active suckles statutes crouchingdays loathing tumult propinqu burrs [PAD] warri springe haters faculty [PAD] [PAD] fowler wa [PAD]isible warrener [PAD] unhousel substantialds louvre bajaz [PAD]leucusued abominsteep 

======= 30.0% generating... =======

[CLS] itern i will, hopes relieving nastybation that gal the 

### With 1000 diffusion steps

In [14]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_1000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)

############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ['life has meanings.', 'call her forth to me.'] ['', '']
RAM used: 2051.57 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 5
})
RAM used: 2051.57 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 5
})
### tokenized_datasets...example [2, 451, 779, 14765, 12, 3]
RAM used: 2051.95 MB


merge and mask:   0%|          | 0/5 [00:00<?, ? examples/s]

RAM used: 2051.98 MB


padding:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 5
}) padded dataset
RAM used: 2051.98 MB
RAM used: 2051.98 MB
### End of reading iteration...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [15]:
word_lst_source, word_lst_recover

(['[CLS] life has meanings. [SEP] [SEP]',
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]"],
 ['[CLS] has, now, have thousand the king, it your wife. [SEP]',
  '[CLS] come love your, you. [SEP]',
  '[CLS] ay, we you lord. [SEP]',
  '[CLS] for, may fear i ever dear [SEP]',
  '[CLS] thou this of to be it a the wife the wife? [SEP]'])

In [16]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

humsogeperance intelligencing turningitimation busied rearward autre schedulesunterique openness trib tiltty [PAD] [PAD] backare grasshoppers sorrowful unple twelvemon smacks continuesowls inv contamille disse usher riotous musics sanctimon wrink salaryesus vouchsprop simplenessrendre abaissez flux [PAD] validityench [PAD] abomin tumbled caust italy bettered pupet pecus [PAD] arbitrating trol delights predominateset move [PAD] barrow nept mows prawns [PAD] scyth shrewdly heigh almanack bricks valueless vulcan enfeebled reproofnker penthesile prague midsum pur iniquities [PAD] [PAD]isen hul immaculate questant penitence [PAD] unfledg glowing packthread restorativematic [PAD] bankruptswine extended agueface loathes hardnessedom fausse vehemency phry [PAD]cer [PAD]ateful manhoods [PAD] waggishdin unbuild ignomin hoodman comfect indentarged courser 

======= 30.0% generating... =======

cromwell creeping hospitality now recre have miscarries the tant pr

### With 2000 diffusion steps

In [18]:
inter_steps=[0.1, 0.3, 0.4, 0.5, 0.7]

word_lst_source, word_lst_recover, word_lst_ref, inter_lst_recover = sampling(best_model, 
                                                           diffusion_2000, 
                                                           tokenizer, 
                                                           data_dir=regular_data_dir, 
                                                           batch_size=10, 
                                                           split='test_custom', 
                                                           seq_len=128,
                                                           inter_steps=inter_steps)


############################## 
Loading text data...
############################## 
Loading dataset from data...
### Loading from the custom TEST set...
### Data samples...
 ["My lord, 'tis the pondering of life's meaning that doth occupy my thoughts most gravely.", 'call her forth to me.'] ['', '']
RAM used: 2056.09 MB
This is raw_datasets:  Dataset({
    features: ['src', 'trg'],
    num_rows: 5
})
RAM used: 2056.09 MB


Running tokenizer on dataset (num_proc=4):   0%|          | 0/5 [00:00<?, ? examples/s]

### tokenized_datasets Dataset({
    features: ['input_id_x', 'input_id_y'],
    num_rows: 5
})
### tokenized_datasets...example [2, 105, 203, 10, 9, 334, 78, 14471, 102, 94, 451, 9, 41, 2679, 110, 457, 19201, 105, 1079, 367, 23847, 12, 3]
RAM used: 2057.09 MB


merge and mask:   0%|          | 0/5 [00:00<?, ? examples/s]

RAM used: 2057.11 MB


padding:   0%|          | 0/5 [00:00<?, ? examples/s]

Dataset({
    features: ['input_id_x', 'input_id_y', 'input_ids', 'input_mask'],
    num_rows: 5
}) padded dataset
RAM used: 2057.16 MB
RAM used: 2057.16 MB
### End of reading iteration...


  0%|          | 0/2000 [00:00<?, ?it/s]

In [19]:
word_lst_source, word_lst_recover

(["[CLS] my lord,'tis the pondering of life's meaning that doth occupy my thoughts most gravely. [SEP] [SEP]",
  '[CLS] call her forth to me. [SEP] [SEP]',
  '[CLS] what is your will? [SEP] [SEP]',
  '[CLS] hark, fair sarah! [SEP] [SEP]',
  "[CLS] what dost thou have planned for the morrow's reprieve? [SEP] [SEP]"],
 ['[CLS] this all of it shall come. my wife we a it, which, [SEP]',
  "[CLS] when'in more you the wife mine, the company it is thus fall, [SEP]",
  '[CLS], i be it shall to. [SEP]',
  '[CLS] without, and, of. [SEP]',
  '[CLS] and me what the wife how in wife,? [SEP]'])

In [20]:
for i in range(len(inter_steps)):
    print(f"======= {inter_steps[i]*100}% generating... =======\n")
    print(inter_lst_recover[i][0], '\n') 

======= 10.0% generating... =======

##fully rendezvous phee unfaithful disguis unat shr catarr embolden wildness mauritania needlessolog langurnityement sickens statusoura [PAD] coriol desiring godfather [PAD] footboygalls viz fillyared courtediating [PAD]aving soo declare scriptures pooped abjur embolden excepted insinuation [PAD] repul obsequ yeomen accomplices unreverent farmer tranqu abilities devises interview merlagon alight prose mondeem masking fount associate hatching alchemist [PAD]tler blackmere [PAD] comedians secondaryaritablyvolence accordant fawns peeps angered couching mithecu sobriety servantmaid [PAD] hardenanth syracusians boil doxy vital thunderbolt rar arraigning partaker chafing [PAD] hales soled staple [PAD] offeringsrases prone [PAD] testimoniedaber quinapalus 

======= 30.0% generating... =======

igius this moveables ofmix wrongly come. my blindness we deathsmenilliades portra which, [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]thum [PAD] 

# Saving embeddings

In [28]:
word_embedding = best_model.word_embedding.weight
# pos_embedding = best_model.position_embeddings.weight

In [29]:
# pos_embedding

In [30]:
# torch.save(pos_embedding, 'pos_embedding.pt')

In [31]:
torch.save(word_embedding, 'word_embedding_with_player.pt')